In [1]:
import pandas as pd
import ast
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('export_sff').getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/21 00:49:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
events = spark.read.csv('Data/events.csv',header=True,inferSchema=True,sep=';')

In [4]:
events_frame = events.filter(events.type == 'Shot').select('id','shot_freeze_frame')

In [5]:
# drop rows with null shot_freeze_frame
events_frame = events_frame.dropna(subset=['shot_freeze_frame'])

In [6]:
events.filter(events.type == 'Shot').select('player','shot_freeze_frame').show(10,False)

+-------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
# the content of the shot_freeze_frame column is a dictionary holding the coordinates of players positions during a shot
# the format is ' [{'location': [x,y], 'player': {'id': id, 'name': name}}, 'position': {'name': name, 'id': id}, 'teammate': True/False},...]
row1 = events_frame.collect()[0].shot_freeze_frame
row2 = events_frame.collect()[482].shot_freeze_frame
#row2 = row2.replace('""Di\'Shon Bernard""',"'Di Shon Bernard'")
#row2 = row2[1:-1]

In [8]:
print(row1)
print(row2)

[{'location': [118.8, 41.9], 'player': {'id': 27225, 'name': 'Camilo Andrés Vargas Gil'}, 'position': {'id': 1, 'name': 'Goalkeeper'}, 'teammate': False}, {'location': [90.9, 57.8], 'player': {'id': 5503, 'name': 'Lionel Andrés Messi Cuccittini'}, 'position': {'id': 22, 'name': 'Right Center Forward'}, 'teammate': True}, {'location': [107.2, 66.0], 'player': {'id': 28263, 'name': 'Gonzalo Ariel Montiel'}, 'position': {'id': 2, 'name': 'Right Back'}, 'teammate': True}, {'location': [106.8, 49.8], 'player': {'id': 28567, 'name': 'Carlos Eccehomo Cuesta Figueroa'}, 'position': {'id': 5, 'name': 'Left Center Back'}, 'teammate': False}, {'location': [106.6, 48.7], 'player': {'id': 2995, 'name': 'Ángel Fabián Di María Hernández'}, 'position': {'id': 12, 'name': 'Right Midfield'}, 'teammate': True}, {'location': [110.1, 43.3], 'player': {'id': 27886, 'name': 'Alexis Mac Allister'}, 'position': {'id': 16, 'name': 'Left Midfield'}, 'teammate': True}, {'location': [91.4, 66.2], 'player': {'id': 

# WE WERE HERE

In [86]:
rows = row2.split('}, {')
a = rows[0].split('},')[0].split(': ')[1].split('],')[0]+']'
b = rows[0].split('},')[1].split(': ')[3].strip("''")
c = rows[0].split('},')[2]
#rows[0].split('},')[0].split('],')[0]+']'

In [91]:
for i in range(0, len(rows)):
    print(ast.literal_eval(rows[i].split('},')[0].split(': ')[1].split('],')[0]+']'), end=', ')
    print(rows[i].split('},')[1].split(': ')[3].strip("''"), end=', ')
    print('True' if 'True' in rows[i].split('},')[2] else 'False')

[116.1, 38.9], Left Center Back, True
[115.4, 33.2], Right Defensive Midfield, True
[116.9, 35.4], Left Back, True
[115.0, 37.6], Center Forward, True
[98.9, 33.5], Right Wing, True
[117.8, 38.1], Left Wing, True
[116.2, 37.9], Center Back, False
[114.8, 35.2], Right Wing, False
[117.9, 36.1], Left Center Back, False
[118.9, 38.4], Goalkeeper, False
[117.0, 39.7], Right Defensive Midfield, False
[117.1, 41.5], Right Center Back, False
[117.1, 42.5], Center Forward, False
[109.3, 34.1], Right Wing Back, False
[117.8, 33.7], Left Wing Back, False
[115.4, 25.6], Left Defensive Midfield, False
[116.4, 25.4], Left Wing, False
[110.1, 15.8], Center Attacking Midfield, True


In [219]:
# Convert to DataFrame
df = pd.DataFrame(row_dict)
# Normalize nested columns
df['player_id'] = df['player'].apply(lambda x: x['id'])
df['player_name'] = df['player'].apply(lambda x: x['name'])
df['position_id'] = df['position'].apply(lambda x: x['id'])
df['position_name'] = df['position'].apply(lambda x: x['name'])
df[['x', 'y']] = pd.DataFrame(df['location'].tolist(), index=df.index)

# Drop the original 'location' column if it's no longer needed
df = df.drop(columns=['location'])
# Drop original nested columns if desired
df = df.drop(columns=['player', 'position'])

# Display the flattened DataFrame
df

,teammate,player_id,player_name,position_id,position_name,x,y
0,True,41593,Willian Joel Pacho Tenorio,5,Left Center Back,116.1,38.9
1,True,37945,Alan Steven Franco Palma,9,Right Defensive Midfield,115.4,33.2
2,True,38004,Piero Martín Hincapié Reyna,6,Left Back,116.9,35.4
3,True,348757,Kevin José Rodríguez Cortez,23,Center Forward,115.0,37.6
4,True,21131,John Yeboah Zamora,17,Right Wing,98.9,33.5
5,True,40211,Jeremy Leonel Sarmiento Morante,21,Left Wing,117.8,38.1
6,False,32685,Joël Latibeaudiere,4,Center Back,116.2,37.9
7,False,31455,Shamar Nicholson,17,Right Wing,114.8,35.2
8,False,4610,Ethan Pinnock,5,Left Center Back,117.9,36.1
9,False,226523,Jahmali Waite,1,Goalkeeper,118.9,38.4


In [ ]:
def extract_shot_ff(df):
    processed_rows = []
    for row in df.collect():
        id = row.id
        shot_frame = row.shot_freeze_frame.split('}, {')
        for i in range(len(shot_frame)):
            x, y = ast.literal_eval(shot_frame[i].split('},')[0].split(': ')[1].split('],')[0] + ']')
            position = shot_frame[i].split('},')[1].split(': ')[3].strip("''")
            teammate = 'True' if 'True' in shot_frame[i].split('},')[2] else 'False'
            processed_rows.append({'Shot_id': id, 'x': x, 'y': y, 'position': position, 'teammate': teammate})
    
    df = pd.DataFrame(processed_rows)
    return df
            
extract_shot_ff(events_frame)

,id,x,y,position,teammate
0,3c3fd192-a115-47bc-9224-b5011bb17885,118.8,41.9,Goalkeeper,False
1,3c3fd192-a115-47bc-9224-b5011bb17885,90.9,57.8,Right Center Forward,True
2,3c3fd192-a115-47bc-9224-b5011bb17885,107.2,66.0,Right Back,True
3,3c3fd192-a115-47bc-9224-b5011bb17885,106.8,49.8,Left Center Back,False
4,3c3fd192-a115-47bc-9224-b5011bb17885,106.6,48.7,Right Midfield,True
...,...,...,...,...,...
797464,614df1f9-b680-4526-a34e-db4bfc1d2496,92.1,40.3,Left Center Forward,True
797465,614df1f9-b680-4526-a34e-db4bfc1d2496,99.6,39.4,Left Center Back,True
797466,614df1f9-b680-4526-a34e-db4bfc1d2496,84.6,34.6,Center Attacking Midfield,True
797467,614df1f9-b680-4526-a34e-db4bfc1d2496,79.1,27.7,Left Back,True
